In [4]:
import os
import csv
import jsonlines
from collections import Counter

In [5]:
import re
import spacy
import random
import json

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use("ggplot")

In [7]:
from random import randint
import xml.etree.ElementTree as ET
from subprocess import Popen, PIPE

In [8]:
nlp = spacy.load("en_core_web_lg")

In [9]:
project_folder='./../Master_Thesis' 
shuffle_num=1

data_path = os.path.join(project_folder, 'Data/thesis_data_shuffle-' + str(shuffle_num) + '_.json')
with open(data_path) as reader:
    data_ = json.load(reader)

In [12]:
data_['metadata']['enrichments'] = {
    'tokenization': {
        'tool': 'spaCy',
        'version': spacy.__version__,
        'key': 'token.text'        
    },
    'part-of-speech': {
        'tool': 'spaCy',
        'version': spacy.__version__,
        'key': 'token.tag_'
    },
    'named-entities': {
        'tool': 'spaCy',
        'version': spacy.__version__,
        'key': 'token.ent_type_'
    },
     'Heidel-Time': {
        'tool': 'Heidel-Time-Tagger',
        'version': 'unknown'
    }
}

In [23]:
def PrepareHeidelTime(sentence, tokens, input_lang='english', TMP_FOLDER_PATH='/tmp/'):
    txt_file_path = os.path.join(TMP_FOLDER_PATH,
                             'time_enrich_tmp_{}.txt'.format(str(randint(10000, 99999))))
    
    with open(txt_file_path, 'w') as txt_file:
    #     for sentence in self.doc.sentences():
        txt_file.write(sentence+ "\n")

    popen_job = Popen(['java', '-jar',
                       'de.unihd.dbs.heideltime.standalone.jar',
                       txt_file_path, '-t', 'NEWS', '-l', input_lang, '-pos', 'no'],
                       stdout=PIPE,
                       cwd='/kuacc/users/simgebasar/heideltime-standalone')
    popen_job.wait()

    os.remove(txt_file_path)
    # Skip the first three lines of the document which is XML definition
    start_lines = ''
    for _ in range(3):
        line = popen_job.stdout.readline()
        line = str(line, 'utf-8')
        start_lines += line

    line = popen_job.stdout.readline()
    line = str(line, 'utf-8')


    # for sentence in self.doc.sentences():
    if '<TIMEX' in line:
        line = start_lines + line + '</TimeML>'
        try:
            time_expr_xml = ET.fromstring(line)
        except ET.ParseError as exp:
            time_expr_xml = None
            print("Error while reading TimeML. Time tags may not occur " \
                           "in the sentence. Traceback: %s", str(exp))
            pass # continue

    if time_expr_xml:
        times = [(time, timetime.attrib['type']) for timetime in time_expr_xml for time in timetime.text.split(' ')]
    else:
        times = []
    time_exprs = []
    for word in tokens:
        if times and word == times[0][0]:
            time_exprs.append((word, times[0][1]))
            times.pop(0)
        else:
            time_exprs.append((word,'0'))
            
    return time_exprs

In [24]:
from pprint import pprint
pprint(data_['metadata'])

{'article_nums': {'test': 32, 'train': 251, 'valid': 32},
 'enrichments': {'Heidel-Time': {'tool': 'Heidel-Time-Tagger',
                                 'version': 'unknown'},
                 'named-entities': {'key': 'token.ent_type_',
                                    'tool': 'spaCy',
                                    'version': '2.0.12'},
                 'part-of-speech': {'key': 'token.tag_',
                                    'tool': 'spaCy',
                                    'version': '2.0.12'},
                 'tokenization': {'key': 'token.text',
                                  'tool': 'spaCy',
                                  'version': '2.0.12'}},
 'label_nums': {'test': {'0': 320, '1': 116, '2': 5},
                'train': {'0': 2472, '1': 1053, '2': 57},
                'valid': {'0': 268, '1': 130, '2': 1}},
 'raw_data_path': '/home/aorus/workspaces/simge/corpus/Sentence/Newindianexpress/20181001-newindianexpress_sentence_classification_adjudicated_20181218

In [25]:
for key, articles in data_['data'].items():
    for article in articles:
        for sent_ind, sentence in enumerate(article['sentences']):
            
            doc = nlp(sentence['sentence'])
            
            tokens = [token.text for token in doc]
            article['sentences'][sent_ind]['tokens'] = tokens 
            article['sentences'][sent_ind]['pos'] = [token.tag_ for token in doc] 
            article['sentences'][sent_ind]['ner'] = [token.ent_type_ if token.ent_iob_ != 'O' else 'O' 
                                                     for token in doc]
            heidel_list = PrepareHeidelTime(sentence['sentence'], tokens)
            article['sentences'][sent_ind]['heidel'] = [word_tag[1] for word_tag in heidel_list]
        

UnboundLocalError: local variable 'time_expr_xml' referenced before assignment

In [ ]:
with open('Data/thesis_data_shuffle-' + str(shuffle_num) + '_enriched_.json', 'w') as outfile:
    json.dump(data_, outfile)

## Wikipedia2Vec

In [ ]:
from wikipedia2vec import Wikipedia2Vec

In [ ]:
MODEL_FILE='Data/enwiki_20180420_500d.pkl.bz2'

In [ ]:
wiki2vec = Wikipedia2Vec.load(MODEL_FILE)

## CHECK 

In [ ]:
import pickle
opt_results_path = 'Scikit-Experiments/Results/results_070919_tfidf_only_mindf_001_maxdf_0_12_numberstoplwords.pickle'

In [ ]:
with open(opt_results_path, 'rb') as file_:
    opt_results = pickle.load(file_)

In [ ]:
opt_results.keys()

## Heidel Time

In [145]:
heidel_list = PrepareHeidelTime(article['sentences'][1]['sentence'], article['sentences'][1]['tokens'])